<a href="https://colab.research.google.com/github/artemcarabash/machine_learning/blob/main/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning. Data Analysis 2
# Text generation with an RNN
## Artiom Carabas; Fatih Karabay

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [2]:
path_to_file = tf.keras.utils.get_file('pride-and-prejudice.txt', 'https://raw.githubusercontent.com/artemcarabash/machine_learning/main/pride-and-prejudice.txt')

### Read the data

First, look in the text:

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 697749 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

Chapter 1


It is a truth universally acknowledged, that a single man in possession
of a good fortune, must be in want of a wife.

However little known the feelings or views of such a man may be on his
first entering a neighbourhood, this trut


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

76 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `preprocessing.StringLookup` layer:

In [7]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

It converts form tokens to character IDs, padding with `0`:

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[52, 53, 54, 55, 56, 57, 58], [75, 76, 77]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the padding and `[UNK]` tokens are set the same way.

In [9]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(697749,), dtype=int64, numpy=array([29, 59, 52, ..., 56, 64, 13])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

C
h
a
p
t
e
r
 
1



In [16]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'C' b'h' b'a' b'p' b't' b'e' b'r' b' ' b'1' b'\r' b'\n' b'\r' b'\n'
 b'\r' b'\n' b'I' b't' b' ' b'i' b's' b' ' b'a' b' ' b't' b'r' b'u' b't'
 b'h' b' ' b'u' b'n' b'i' b'v' b'e' b'r' b's' b'a' b'l' b'l' b'y' b' '
 b'a' b'c' b'k' b'n' b'o' b'w' b'l' b'e' b'd' b'g' b'e' b'd' b',' b' '
 b't' b'h' b'a' b't' b' ' b'a' b' ' b's' b'i' b'n' b'g' b'l' b'e' b' '
 b'm' b'a' b'n' b' ' b'i' b'n' b' ' b'p' b'o' b's' b's' b'e' b's' b's'
 b'i' b'o' b'n' b'\r' b'\n' b'o' b'f' b' ' b'a' b' ' b'g' b'o' b'o' b'd'
 b' ' b'f' b'o' b'r'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [18]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Chapter 1\r\n\r\n\r\nIt is a truth universally acknowledged, that a single man in possession\r\nof a good for'
b'tune, must be in want of a wife.\r\n\r\nHowever little known the feelings or views of such a man may be o'
b'n his\r\nfirst entering a neighbourhood, this truth is so well fixed in the minds\r\nof the surrounding f'
b'amilies, that he is considered the rightful property\r\nof some one or other of their daughters.\r\n\r\n"My'
b' dear Mr. Bennet," said his lady to him one day, "have you heard that\r\nNetherfield Park is let at las'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Chapter 1\r\n\r\n\r\nIt is a truth universally acknowledged, that a single man in possession\r\nof a good fo'
Target: b'hapter 1\r\n\r\n\r\nIt is a truth universally acknowledged, that a single man in possession\r\nof a good for'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [24]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 78) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  19968     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  79950     
Total params: 4,038,222
Trainable params: 4,038,222
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [30]:
sampled_indices

array([20, 22, 51, 31, 62,  1, 42, 15, 57,  2, 34, 17, 11, 46, 48, 54, 58,
       33, 26, 77, 23, 69, 51, 52, 23,  7, 41, 67, 65, 70, 38, 75,  2, 69,
        4, 27,  5, 10, 45, 46, 31, 19, 63, 25, 13, 18, 18,  3, 37, 67, 28,
       76, 37,  6,  6, 40, 58, 43, 58, 60,  6, 75, 39, 67,  1, 51, 37, 52,
       10, 44,  8, 24, 27, 12, 74, 43,  7, 12, 28, 36, 14, 23, 68, 51, 34,
       56, 74, 59, 11, 56, 18, 16, 38, 17, 68, 42, 45,  6,  1, 47])

Decode these to see the text predicted by this untrained model:

In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' relations, with only\r\none serious, parting look, went away.\r\n\r\nAs he quitted the room, Elizabeth fe'

Next Char Predictions:
 b'68_Ek[UNK]P1f\nH3,UWcgG?z9r_a9\'OpnsLx\nr A!*TUE5l;.44\rKpByK""NgRgi"xMp[UNK]_Ka*S(:A-wR\'-BJ09q_Hewh,e42L3qPT"[UNK]V'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 78)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.356842


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [34]:
tf.exp(mean_loss).numpy()

78.01039

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [35]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [36]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [37]:
EPOCHS = 20

In [38]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
107/107 [==============================] - 8s 57ms/step - loss: 3.4253
Epoch 2/20
107/107 [==============================] - 7s 58ms/step - loss: 2.1914
Epoch 3/20
107/107 [==============================] - 7s 59ms/step - loss: 1.8482
Epoch 4/20
107/107 [==============================] - 7s 59ms/step - loss: 1.5726
Epoch 5/20
107/107 [==============================] - 7s 58ms/step - loss: 1.3956
Epoch 6/20
107/107 [==============================] - 7s 57ms/step - loss: 1.2785
Epoch 7/20
107/107 [==============================] - 7s 57ms/step - loss: 1.2030
Epoch 8/20
107/107 [==============================] - 7s 57ms/step - loss: 1.1471
Epoch 9/20
107/107 [==============================] - 7s 56ms/step - loss: 1.1010
Epoch 10/20
107/107 [==============================] - 7s 56ms/step - loss: 1.0546
Epoch 11/20
107/107 [==============================] - 7s 56ms/step - loss: 1.0147
Epoch 12/20
107/107 [==============================] - 7s 56ms/step - loss: 0.9756
Epoch 13/20
1

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [39]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['', '[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [40]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [41]:
start = time.time()
states = None
next_char = tf.constant(['Generated Text:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Generated Text:

Elizabeth saw that of _this_ we are all somewiment over my two most guarded and sigh
of seeing Jane, she added:
"Lizzy, I have," said Jane, "that mine would not ake, could bear such a favourite; one, observing that let de Bourgh, and taking the obligation must
belonging to orrout me dow."

"Oh! by in that variety of this house
un. Neversonly see him in his fear. Shanly a gave way, and gave them together, which
both she added in distressed
constant communication. It ore to the same time. It is every mind,
"It might attention to visit his wife a gentleman did not advantage. But poor Meryton? Oh!
why, in silence with Elizabeth 
grieved for his patroness.

They better not very fan by themselves; and I did not listen, they friends and breakfast, took
the necessary terms over. Elizabeth then reached to confirm her. What are her fiture beyond to part over him again.
Half when she left Netherfield believing it. Well, if I have had to see you
at Pemberley, eye, and-the man we 


The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [42]:
start = time.time()
states = None
next_char = tf.constant(['Generated Text:', 'Generated Text:', 'Generated Text:', 'Generated Text:', 'Generated Text:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'Generated Text:--\r\n\r\n"But your father has no best cordial feelings, and only the first sentence alarming on the perpossible match, or occasion, was\r\nnot seen a very propition which their first object was never so prevailed on\r\none. An end or carriage? As a chill protee that were larger to be the carriage?"\r\n\r\nShe then exclaimed, "but I am very serious like of her curiosity; he was allowed at fir the\r\nentrance of Bingley, and she turned her interest; for who, in a moment which she felt a smile at suspense,\r\nand Mr. Denny and his sonsisting more surprised before the dance with Mr. Wickham, and\r\nintroduction at long, he was lagged before the intention maintains, expressively the\r\nshopled better offered her eetrest disposition.\r\n\r\nIt cannot understand it, I could hardly hulf home. Mrs. Gardiner regained her gown. She lamed at\r\nbr the exercise of tea-ten thousand pounds, you was it difficult on terms.\r\nThis was now patience to accept the dance, and 

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [43]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [44]:
states = None
next_char = tf.constant(['Generated Text:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Generated Text:--Merice, really
repressed."

"A great deal more than _yours_ silence, when she read. Oh! brage s


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [45]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [46]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [47]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [48]:
model.fit(dataset, epochs=1)

107/107 [==============================] - 13s 106ms/step - loss: 2.8803


Or if you need more control, you can write your own complete custom training loop:

In [49]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.2837
Epoch 1 Batch 50 Loss 2.1464
Epoch 1 Batch 100 Loss 1.9971

Epoch 1 Loss: 2.1193
Time taken for 1 epoch 12.23 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.9429
Epoch 2 Batch 50 Loss 1.8356
Epoch 2 Batch 100 Loss 1.6687

Epoch 2 Loss: 1.7960
Time taken for 1 epoch 11.91 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6579
Epoch 3 Batch 50 Loss 1.5514
Epoch 3 Batch 100 Loss 1.4544

Epoch 3 Loss: 1.5508
Time taken for 1 epoch 11.87 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4252
Epoch 4 Batch 50 Loss 1.3652
Epoch 4 Batch 100 Loss 1.2843

Epoch 4 Loss: 1.3827
Time taken for 1 epoch 11.81 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 1.2892
Epoch 5 Batch 50 Loss 1.2735
Epoch 5 Batch 100 Loss 1.2576

Epoch 5 Loss: 1.2753